这个算法个人不是很了解，也是根据视频和图书总结的

使用一个函数来计算存储地址，并且将这个存储到哈希表格

1. 取余法:
    1. 关键公式： `index = number % num`
    2. 假设数组 `[13, 17, 19, 26, 30]` 这里`num`先取`5`
    3. 13 % 5 余 3， 17 % 5 余 2， 19 % 5 余 4， 26 % 5 余 1， 30 % 5 余 0
    4. 得出表格
| 0  | 1  | 2  | 3  | 4  |
|----|----|----|----|----|
| 30 | 26 | 17 | 13 | 19 |

2. 折叠法
    1. 如果数组为 `[13, 17, 19, 26, 30, 40]`, 那么就会得到两个余数为0的
    2. 假设一个数字： 12345678，将他们两个为一组相加 `12 + 34 + 56 + 78 = 180`, `num` 还取 `5` 则 0 对应数字 12345678

3. 平方取中法
    1.   同样以 `[13, 17, 19, 26, 30]` 为例， 对每个数进行平方
    2. `[169, 289, 361, 676, 900]` 取一位或者两位中间数字， 这里演示两位， 得： [16, 28, 36, 67, 90], 这里 `num` 取 13
    3. 得 [3, 2, 7, 4, 2, 2]
    4. 这样会造成碰撞和溢出，emm，FUCK！！！

In [ ]:
class HashMap:
    def __init__(self):
        self.hash_size = 13
        self.throw = [None] * self.hash_size  # 存储key
        self.data = [None] * self.hash_size  # 存储value

    def remainder(self, dividend, divisor):
        return dividend % divisor

    def rehash(self, dividend, divisor):
        return (dividend + 1) % divisor

    def put(self, key, value):
        hash_value = self.remainder(key, len(self.throw))  # 计算位置
        if self.throw[hash_value] is None:  # 如果当前key表中hash_value位置没有数据
            self.throw[hash_value] = key
            self.data[hash_value] = value
        else:
            if self.throw[hash_value] == key:  # 当有 H[15] = "田" H[15] = "文" 两个时，会走到这个分支
                self.data[hash_value] = value
            else:  # 当有 H[15] = "田" H[28] = "妈" 时，会走到这个分支
                next_slot = self.rehash(hash_value, len(self.throw))  # 将self.remainder(key, len(self.throw))计算的结果 + 1， 重新计算位置
                while self.throw[next_slot] is not None and next_slot != key: # 重新计算位置如果有数据，那就再次 +1
                    next_slot = self.rehash(next_slot, len(self.throw))
                if self.throw[next_slot] is None:
                    self.throw[next_slot] = key
                    self.data[next_slot] = value
                else:
                    self.data[next_slot] = value

    def get(self, key):
        start_slot = self.remainder(key, len(self.throw))
        data = None
        is_found = False
        stop = False
        pos = start_slot
        while pos is not None and not is_found and not stop:
            if self.throw[pos] == key:
                is_found = True
                data = self.data[pos]
            else:
                pos = self.rehash(pos, len(self.throw))
                if pos == start_slot:
                    stop = True
        return data

    def __getitem__(self, item):
        return self.get(item)

    def __setitem__(self, key, value):
        self.put(key, value)


H = HashMap()
H[15] = "田"
H[28] = "妈"
H[19] = "文"
H[23] = "镜"
H[26] = "我"
H[33] = "操"
H[47] = "你"

print(H.throw)
print(H.data)
print(H[15])

### Python `__getitem__` 和 `__setitem__` 详细解析

在 Python 中，`__getitem__` 和 `__setitem__` 是 **魔法方法（Magic Methods）**，用于重载 `[]` 操作符，使自定义对象像列表、字典一样支持索引访问和赋值操作。

---

## 1. `__getitem__(self, key)`  —— 获取值
当你使用 `obj[key]` 访问元素时，Python 会调用 `obj` 所在类的 `__getitem__` 方法。

### **基本用法**
```python
class MyDict:
    def __init__(self, data):
        self.data = data  # 使用字典存储数据

    def __getitem__(self, key):
        print(f"获取 {key}")
        return self.data[key]

my_dict = MyDict({"a": 1, "b": 2})
print(my_dict["a"])  # 触发 __getitem__
```

**输出：**
```
获取 a
1
```

🔹 **解释**：
- `my_dict["a"]` 触发 `__getitem__("a")` 方法，返回 `self.data["a"]`。
- 你可以在 `__getitem__` 里 **添加日志、权限控制、缓存逻辑等**。

---

## 2. `__setitem__(self, key, value)`  —— 设置值
当你执行 `obj[key] = value` 时，Python 调用 `__setitem__` 进行赋值操作。

### **基本用法**
```python
class MyDict:
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data.get(key, None)  # 如果 key 不存在，返回 None

    def __setitem__(self, key, value):
        print(f"设置 {key} 为 {value}")
        self.data[key] = value  # 存储值

my_dict = MyDict()
my_dict["x"] = 42  # 触发 __setitem__
print(my_dict["x"])  # 触发 __getitem__
```

**输出：**
```
设置 x 为 42
42
```

🔹 **解释**：
- `my_dict["x"] = 42` 触发 `__setitem__("x", 42)`，在 `self.data` 字典中存入 `{"x": 42}`。
- `print(my_dict["x"])` 触发 `__getitem__("x")`，返回 `42`。

---

## 3. 支持 **切片（slice）**
如果 `key` 传入的是一个 `slice` 对象（如 `obj[1:4]`），`__getitem__` 需要特殊处理。

### **示例**
```python
class MyList:
    def __init__(self, data):
        self.data = data

    def __getitem__(self, key):
        if isinstance(key, slice):  # 处理切片
            print(f"获取切片 {key}")
            return self.data[key.start:key.stop:key.step]
        return self.data[key]

my_list = MyList([0, 1, 2, 3, 4, 5])
print(my_list[1:4])  # 触发 __getitem__，返回 [1, 2, 3]
```

**输出：**
```
获取切片 slice(1, 4, None)
[1, 2, 3]
```

🔹 **解释**：
- `obj[1:4]` 传入的 `key` 是 `slice(1, 4, None)`，表示索引 `1, 2, 3` 的元素。
- `self.data[key.start:key.stop:key.step]` 取出 `[1, 2, 3]` 并返回。

---

## 4. 结合 `__delitem__`（删除元素）
除了 `__getitem__` 和 `__setitem__`，Python 还提供了 `__delitem__` 让 `del obj[key]` 生效。

### **示例**
```python
class MyDict:
    def __init__(self, data):
        self.data = data

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        print(f"删除 {key}")
        del self.data[key]

my_dict = MyDict({"a": 1, "b": 2, "c": 3})
del my_dict["b"]  # 触发 __delitem__
print(my_dict.data)
```

**输出：**
```
删除 b
{'a': 1, 'c': 3}
```

🔹 **解释**：
- `del my_dict["b"]` 触发 `__delitem__("b")`，删除 `self.data["b"]`。
- 现在 `my_dict.data` 只剩下 `{"a": 1, "c": 3}`。

---

## 5. 应用场景
| **场景** | **用法** |
|----------|---------|
| **模拟字典** | 让自定义类像字典一样存取数据 |
| **模拟列表** | 让自定义对象支持索引、切片操作 |
| **缓存机制** | 在 `__getitem__` 里增加缓存逻辑 |
| **访问控制** | 在 `__setitem__` 里添加权限验证 |
| **数据格式化** | 统一处理输入、输出数据格式 |

---

## 6. **完整示例：增强版 `MyDict`**
```python
class MyDict:
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        if key in self.data:
            print(f"获取 {key}: {self.data[key]}")
            return self.data[key]
        else:
            print(f"Key '{key}' 不存在！")
            return None

    def __setitem__(self, key, value):
        print(f"设置 {key} 为 {value}")
        self.data[key] = value

    def __delitem__(self, key):
        if key in self.data:
            print(f"删除 {key}")
            del self.data[key]
        else:
            print(f"Key '{key}' 不存在，无法删除！")

# 测试
my_dict = MyDict()
my_dict["name"] = "Alice"
print(my_dict["name"])
del my_dict["name"]
print(my_dict["name"])
```

**输出**
```
设置 name 为 Alice
获取 name: Alice
Alice
删除 name
Key 'name' 不存在！
None
```

🔹 **增强点**：
- 访问时提示是否 `key` 存在，避免 `KeyError`。
- 删除时检查 `key` 是否存在。

---

## 7. **总结**
| 方法 | 作用 | 触发方式 |
|------|------|----------|
| `__getitem__(self, key)` | 获取值 | `obj[key]` |
| `__setitem__(self, key, value)` | 设置值 | `obj[key] = value` |
| `__delitem__(self, key)` | 删除值 | `del obj[key]` |
| `__getitem__` 处理 `slice` | 处理切片 | `obj[start:end]` |
